In [4]:
!PYTHONPATH=../kari

In [5]:
import numpy as np

from sequence_processing_model import SequenceProcessingModel
from dataset import Dataset
from utils_parameter_parsing import *

from sklearn.metrics import precision_recall_fscore_support

ModuleNotFoundError: No module named 'sequence_processing_model'

Load the parameters from the config file and create a model.

In [2]:
config = config_parser('./config.ini') # parse config.ini
parameters = process_parameters(config, {}) # process_parameters expects a dictionary

sequence_processing_model = SequenceProcessingModel(**parameters)

Loading dataset... 

Done (0.81 seconds)
Loading embeddings... found 1891782 word vectors of dimension 200.
Done (113.74 seconds)
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 200)           2164000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 50, 200)           240800    
___________________________________________

Train the model

In [3]:
sequence_processing_model.fit()

Train on 5712 samples, validate on 635 samples
Epoch 1/50
5712/5712 [==============================] - 392s 69ms/step - loss: 3.2145 - acc: 0.9120 - val_loss: 2.6345 - val_acc: 0.9243
Epoch 2/50
5712/5712 [==============================] - 388s 68ms/step - loss: 3.0998 - acc: 0.9277 - val_loss: 2.5826 - val_acc: 0.9399
Epoch 3/50
5712/5712 [==============================] - 388s 68ms/step - loss: 3.0591 - acc: 0.9401 - val_loss: 2.5517 - val_acc: 0.9542
Epoch 4/50
5712/5712 [==============================] - 392s 69ms/step - loss: 3.0344 - acc: 0.9485 - val_loss: 2.5344 - val_acc: 0.9576
Epoch 5/50
5712/5712 [==============================] - 388s 68ms/step - loss: 3.0173 - acc: 0.9526 - val_loss: 2.5216 - val_acc: 0.9599
Epoch 6/50
5712/5712 [==============================] - 379s 66ms/step - loss: 3.0047 - acc: 0.9567 - val_loss: 2.5129 - val_acc: 0.9610
Epoch 7/50
5712/5712 [==============================] - 367s 64ms/step - loss: 2.9952 - acc: 0.9602 - val_loss: 2.5103 - val_acc: 0

,acc,loss,val_acc,val_loss
0,0.912048,3.214479,0.924322,2.634516
1,0.927744,3.099762,0.939908,2.582646
2,0.940103,3.059051,0.954207,2.551660
3,0.948495,3.034433,0.957628,2.534413
4,0.952603,3.017336,0.959899,2.521640
5,0.956655,3.004738,0.961038,2.512852
6,0.960181,2.995198,0.960332,2.510276
7,0.961652,2.987671,0.961883,2.502305
8,0.964205,2.981507,0.962621,2.497823
9,0.965516,2.976375,0.964829,2.492595


In [4]:
pred_idx, gold_idx, labels_ = sequence_processing_model.predict()

0.9853404255319149


In [5]:
np.count_nonzero((pred_idx == gold_idx)) / len(pred_idx)
# gold_idx[:18]

0.9853404255319149

In [6]:
sequence_processing_model.ds.tag_type_to_index

{'B-Disease': 2, 'E-Disease': 0, 'I-Disease': 1, 'O': 4, 'S-Disease': 3}

In [7]:
neg_tag = sequence_processing_model.ds.tag_type_to_index['O']

In [8]:
gold_labels = gold_idx != neg_tag
gold_labels[:18]

array([False, False, False, False, False, False,  True,  True,  True,
       False, False, False,  True,  True,  True,  True,  True, False])

In [9]:
pred_labels = pred_idx != neg_tag
pred_labels[:18]

array([False, False, False, False, False, False,  True,  True,  True,
       False, False, False,  True,  True,  True,  True,  True, False])

In [10]:
TP = 0
FP = 0
FN = 0

for pred, gold in zip(pred_labels, gold_labels):
    # FN
    if gold and not pred:
        FN += 1
    # FP
    if not gold and pred:
        FP += 1
    # TP
    if gold and pred:
        TP += 1
        
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 = 2 * precision * recall / (precision + recall)
        
    

In [11]:
TP, FP, FN

(1629, 23243, 399)

In [12]:
precision, recall, f1

(0.0654953361209392, 0.8032544378698225, 0.12111524163568772)

# Testing

Create a model for testing

In [ ]:
config = config_parser('./config.ini') # parse config.ini
parameters = process_parameters(config, {}) # process_parameters expects a dictionary

sequence_processing_model = SequenceProcessingModel(**parameters)

In [ ]:
# sequence_processing_model._load_embeddings()